![alt text](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logo/0x150/1643104191/logo-mse.png "MSE Logo") 
![alt text](https://www.hes-so.ch/typo3conf/ext/wng_site/Resources/Public/HES-SO/img/logo_hesso_master_tablet.svg "Hes Logo")

# Auteur : Abdi VURAL
## L'objectif
Développer des modèles de prompts pour guider ChatGPT dans l'analyse des clusters de séries temporelles

### Clustering et Analyse des Séries Temporelles avec Interaction LLM

Ce script réalise le clustering des données de séries temporelles de températures mensuelles, génère des descriptions des clusters et utilise un modèle de langage pour interpréter les tendances et implications des résultats.

In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans
import os
from dotenv import load_dotenv
import requests

# Load environment variables
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Load the time series data
file_path = "C:/Users/Abdi/Desktop/data/raw/meteo_idaweb.csv"
data = pd.read_csv(file_path)

# Preprocess the data
data['date'] = pd.to_datetime(data['year'] * 1000 + data['day_of_year'], format='%Y%j') + pd.to_timedelta(data['minute_of_day'], unit='m')
data.set_index('date', inplace=True)

# Extract monthly average temperature
data['month'] = data.index.month
monthly_avg_temp = data.groupby(['name', 'month'])['tre200s0'].mean().reset_index()
pivot_monthly_avg_temp = monthly_avg_temp.pivot(index='name', columns='month', values='tre200s0')
pivot_monthly_avg_temp_filled = pivot_monthly_avg_temp.fillna(pivot_monthly_avg_temp.mean())

# Convert data to time series dataset for clustering
formatted_dataset = to_time_series_dataset(pivot_monthly_avg_temp_filled.to_numpy())

# Perform time series clustering
n_clusters = 4
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", random_state=33)
labels = model.fit_predict(formatted_dataset)
pivot_monthly_avg_temp_filled['cluster'] = labels

# Generate cluster descriptions
def generate_cluster_descriptions(clustered_data, n_clusters):
    descriptions = []
    for cluster in range(n_clusters):
        cluster_data = clustered_data[clustered_data['cluster'] == cluster].drop('cluster', axis=1)
        mean_temp = cluster_data.mean(axis=1).mean()
        std_temp = cluster_data.mean(axis=1).std()
        descriptions.append(f"Cluster {cluster} has an average temperature of {mean_temp:.2f}°C with a standard deviation of {std_temp:.2f}°C.")
    return descriptions

# Example usage
cluster_descriptions = generate_cluster_descriptions(pivot_monthly_avg_temp_filled, n_clusters)

# Prompting the LLM
def call_chatgpt(prompt):
    if not api_key:
        raise ValueError("API key is not set. Please set the OPENAI_API_KEY environment variable.")
    
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    payload = {
        'model': 'gpt-3.5-turbo',  # or 'gpt-4' if available
        'messages': [{
            'role': 'user',
            'content': prompt
        }]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error {response.status_code}: {response.text}"

# Create the prompt for the LLM
prompt = f"""
We have identified several clusters in the temperature data:
{', '.join(cluster_descriptions)}

Based on these cluster descriptions, can you provide an interpretation of the overall temperature trends and potential implications for future weather patterns?
"""

# Get the response from the LLM
response = call_chatgpt(prompt)
print(response)


Based on the identified clusters, it appears that there are distinct temperature patterns within the dataset. Cluster 1 has the lowest average temperature and highest standard deviation, indicating that this cluster likely represents colder and more variable temperature conditions. Cluster 0 and Cluster 2 have slightly higher average temperatures and lower standard deviations, suggesting more moderate and stable temperature conditions. Cluster 3 has the highest average temperature among the clusters, indicating warmer temperatures.

These temperature clusters could potentially represent different weather patterns or seasons within the dataset. For example, Cluster 1 may represent winter or colder months, while Cluster 3 may represent summer or warmer months. The differences in average temperatures and standard deviations among the clusters could have implications for future weather patterns. For instance, if temperatures in Cluster 3 (warmer temperatures) are increasing over time, this

In [3]:
questions = [
    "Can you describe the main characteristics of each temperature cluster?",
    "What are the key temperature trends observed over the past years?",
    "Are there any significant anomalies in the temperature data? Which clusters do they belong to?",
    "Based on current trends, what are the projected temperature trends for the next year?",
    "How might the predicted temperature trends impact future weather patterns?",
    "How can LLMs be guided by clustering data to provide more intuitive interpretations of temporal data?"
]
for question in questions:
    print(f"Question: {question}")
    response = call_chatgpt(question)
    print(response)
    print("\n")

Question: Can you describe the main characteristics of each temperature cluster?
Certainly! The main characteristics of each temperature cluster typically include:

1. Cold Cluster: This cluster typically consists of temperatures that are below average for the region and time of year. Cold clusters often feature temperatures that are chilly or downright frigid, with a higher likelihood of snow and ice.

2. Mild Cluster: The mild cluster usually comprises temperatures that are moderate and comfortable, neither too hot nor too cold. This cluster is often associated with pleasant weather conditions and may be characterized by mild breezes and moderate levels of precipitation.

3. Warm Cluster: The warm cluster typically includes temperatures that are above average for the region and time of year. This cluster tends to be associated with hot and sunny weather, with higher temperatures and lower humidity levels.

4. Hot Cluster: The hot cluster generally features temperatures that are well 